In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
from matplotlib.ticker import FuncFormatter


# Formatear las etiquetas del eje y
formatter = FuncFormatter(lambda x, pos: '{:,.0f}'.format(x))

np.random.seed(42)

Leemos el archivo de excel

In [3]:
df = pd.concat([
    pd.read_excel(f)
    for f in glob("/content/drive/MyDrive/Universidad/papers/gender_bias/augmented_data/*.xlsx")
]).reset_index(drop=True)
print(df.shape, df["index"].nunique())
df.head(2)

(16000, 15) 16000


,index,titulo,url,mensaje,fecha,frases,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,potencialmente_sesgable_otro_sesgo,revisado,description_bias_removal,biased_words,unbiased_words
0,10000,Continuación del Semestre Otoño 2020 FCFM,detalle?id=31470,"Estimada Comunidad, \n\n\n\nLa situación extr...",2020-04-03 17:19:03,- El Semestre Otoño 2020 terminará el 1 de ago...,NaN,NaN,NaN,NaN,No,Si,NaN,NaN,NaN
1,10001,Continuación del Semestre Otoño 2020 FCFM,detalle?id=31470,"Estimada Comunidad, \n\n\n\nLa situación extr...",2020-04-03 17:19:03,- Los estudiantes tendrán dos instancias para ...,Si,No,- Los estudiantes tendrán dos instancias para ...,Las/os estudiantes tendrán dos instancias para...,No,Si,"Se incluyó la forma femenina ""las"" junto a la ...",estudiantes,las/os estudiantes


In [4]:
# prompt: get the field fecha and reorder the dataframe in a  copy from older to newest

# Create a copy of the DataFrame to avoid modifying the original
df_copy = df.copy()

# Convert 'fecha' column to datetime objects if it's not already
if not pd.api.types.is_datetime64_any_dtype(df_copy['fecha']):
    df_copy['fecha'] = pd.to_datetime(df_copy['fecha'])

# Sort the DataFrame by the 'fecha' column in ascending order (oldest to newest)
df_sorted = df_copy.sort_values(by='fecha')

# Display the sorted DataFrame (optional)
print(df_sorted.head())

       index                                             titulo  \
11999  23999                          Charla 1 Haciendo Empresa   
11997  23997  Concierto en el Teatro Municipal ¡ENTRADAS DIS...   
11996  23996  Concierto en el Teatro Municipal ¡ENTRADAS DIS...   
11998  23998  Concierto en el Teatro Municipal ¡ENTRADAS DIS...   
11995  23995                    Charla Abierta del Curso EH2507   

                   url                                            mensaje  \
11999  detalle?id=4893  Tema: Los Desafíos del Liderazgo: El caso Walm...   
11997  detalle?id=4897  AÚN QUEDAN ENTRADAS DISPONIBLES para el concie...   
11996  detalle?id=4897  AÚN QUEDAN ENTRADAS DISPONIBLES para el concie...   
11998  detalle?id=4897  AÚN QUEDAN ENTRADAS DISPONIBLES para el concie...   
11995  detalle?id=4913  Afiche: \n\nwww.u-cursos.cl/ ... Carlos_Huneeu...   

                    fecha                                             frases  \
11999 2012-08-06 14:16:06  Tema: Los Desafíos del Lide

In [5]:
df_sorted.tail()

,index,titulo,url,mensaje,fecha,frases,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,potencialmente_sesgable_otro_sesgo,revisado,description_bias_removal,biased_words,unbiased_words
7001,1,Revisa actualización Resultados MINEDUC 2023,detalle?id=69349,Estimad@s Estudiantes: Recordamos que se liber...,2023-08-07 15:19:07,Quienes no queden conformes con estos resultad...,No,No,Los que no queden conformes con estos resultad...,Quienes no queden conformes con estos resultad...,Si,Si,"Se reemplazó el pronombre ""Los"" (masculino plu...",Los,Quienes
7000,0,Revisa actualización Resultados MINEDUC 2023,detalle?id=69349,Estimad@s Estudiantes: Recordamos que se liber...,2023-08-07 15:19:07,Estimad@s Estudiantes: Recordamos que se liber...,No,No,Estimados Estudiantes: Recordamos que se liber...,Estimad@s Estudiantes: Recordamos que se liber...,No,Si,"Se añadió la arroba (@) a ""Estimados"" para inc...",Estimados,Estimad@s
7006,6,Revisa actualización Resultados MINEDUC 2023,detalle?id=69349,Estimad@s Estudiantes: Recordamos que se liber...,2023-08-07 15:19:07,Este documento y los plazos respectivos son in...,NaN,NaN,NaN,NaN,No,Si,NaN,NaN,NaN
7005,5,Revisa actualización Resultados MINEDUC 2023,detalle?id=69349,Estimad@s Estudiantes: Recordamos que se liber...,2023-08-07 15:19:07,"*Si tienes asignación o renovación de FSCU, re...",NaN,NaN,NaN,NaN,Si,Si,NaN,NaN,NaN
7004,4,Revisa actualización Resultados MINEDUC 2023,detalle?id=69349,Estimad@s Estudiantes: Recordamos que se liber...,2023-08-07 15:19:07,*Si tienes asignación de Gratuidad y pagaste e...,NaN,NaN,NaN,NaN,Si,Si,NaN,NaN,NaN


In [6]:
# prompt: i want to get the dataframe of all the fields that fall outside of this dates:  23 August 2023 and 6 December 2023. use the correct format.

# Define the start and end dates
start_date = pd.to_datetime('2023-08-23')
end_date = pd.to_datetime('2023-12-06')

# Filter the DataFrame to include rows outside the specified date range
df_outside_range = df_sorted[(df_sorted['fecha'] < start_date)]

# Display the resulting DataFrame
df_outside_range.head()

,index,titulo,url,mensaje,fecha,frases,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,potencialmente_sesgable_otro_sesgo,revisado,description_bias_removal,biased_words,unbiased_words
11999,23999,Charla 1 Haciendo Empresa,detalle?id=4893,Tema: Los Desafíos del Liderazgo: El caso Walm...,2012-08-06 14:16:06,Tema: Los Desafíos del Liderazgo: El caso Walm...,NaN,NaN,NaN,NaN,NO,SI,NaN,NaN,NaN
11997,23997,Concierto en el Teatro Municipal ¡ENTRADAS DIS...,detalle?id=4897,AÚN QUEDAN ENTRADAS DISPONIBLES para el concie...,2012-08-06 16:52:06,"Sala Principal, Teatro Municipal de Santiago.",NaN,NaN,NaN,NaN,NO,SI,NaN,NaN,NaN
11996,23996,Concierto en el Teatro Municipal ¡ENTRADAS DIS...,detalle?id=4897,AÚN QUEDAN ENTRADAS DISPONIBLES para el concie...,2012-08-06 16:52:06,AÚN QUEDAN ENTRADAS DISPONIBLES para el concie...,NaN,NaN,NaN,NaN,NO,SI,NaN,NaN,NaN
11998,23998,Concierto en el Teatro Municipal ¡ENTRADAS DIS...,detalle?id=4897,AÚN QUEDAN ENTRADAS DISPONIBLES para el concie...,2012-08-06 16:52:06,"Retirar entradas en: Área de Comunicaciones, T...",NaN,NaN,NaN,NaN,NO,SI,NaN,NaN,NaN
11995,23995,Charla Abierta del Curso EH2507,detalle?id=4913,Afiche: \n\nwww.u-cursos.cl/ ... Carlos_Huneeu...,2012-08-09 11:12:09,Afiche: www.u-cursos.cl/ . Carlos_Huneeus-1.jpg,NaN,NaN,NaN,NaN,NO,SI,NaN,NaN,NaN


In [7]:
df_outside_range.columns

Index(['index', 'titulo', 'url', 'mensaje', 'fecha', 'frases',
       'sesgo_pronombre', 'sesgo_otro', 'version_con_sesgo',
       'version_sin_sesgo', 'potencialmente_sesgable_otro_sesgo', 'revisado',
       'description_bias_removal', 'biased_words', 'unbiased_words'],
      dtype='object')

In [8]:
# prompt: count the number of instances where the output is Nan in the column version_sin_sesgo. and the number of instances when there is a message.

# Count NaN values in 'version_sin_sesgo'
nan_count = df_outside_range['version_sin_sesgo'].isna().sum()
print(f"Number of NaN values in 'version_sin_sesgo': {nan_count}")

# Assuming 'message' column exists and contains strings.
# If 'message' contains NaN, adjust the code accordingly.
message_count = df_outside_range['version_sin_sesgo'].notna().sum() #Counts non-null values
print(f"Number of instances with a message: {message_count}")

Number of NaN values in 'version_sin_sesgo': 11417
Number of instances with a message: 4583


In [9]:
# prompt: the original dataframe, create a train and validation split of 90-10, with a set random seed for reproducibility.

from sklearn.model_selection import train_test_split

# Assuming df_outside_range is your DataFrame
# Split the data into training and validation sets
train_df, val_df = train_test_split(
    df_sorted,
    test_size=0.1,
    random_state=36 # Set random seed for reproducibility
)

print(f"Training set shape: {train_df.shape}")
print(f"Validation set shape: {val_df.shape}")

Training set shape: (14400, 15)
Validation set shape: (1600, 15)


In [10]:
message_count = val_df['version_sin_sesgo'].notna().sum() #Counts non-null values
print(f"Number of instances with a message in validation: {message_count}")

Number of instances with a message in validation: 443


In [11]:
# prompt: save the train and validation dataset splits as xlsx files in a new folder inside this directory: /content/drive/MyDrive/Universidad/papers/gender_bias

import os

# Create the new folder if it doesn't exist
folder_path = "/content/drive/MyDrive/Universidad/papers/gender_bias/train_val_splits"
os.makedirs(folder_path, exist_ok=True)

# Save the training set to an xlsx file
train_file_path = os.path.join(folder_path, "train_data.xlsx")
train_df.to_excel(train_file_path, index=False)

# Save the validation set to an xlsx file
val_file_path = os.path.join(folder_path, "val_data.xlsx")
val_df.to_excel(val_file_path, index=False)